# let's get some dockets
- and maybe get the case titles and docket entries of documents that are search warrants?
    - make a classifier operating on descritions to exclude docket entries that are ABOUT search
     warrants but which aren't a search warrant in and of itself
    - NER
- ALSO find witness lists, maybe?
- finding things that involve facial recognition
- extracting dollar amounts from fraud cases
- find named entities in common, as defendents/witnesses in several similar lawsuits
- find cases sent to arbitration in California
- identify interesting cases (those mentioned in news articles) (see "case number" search from Media Cloud in downloads folder)

##### LEGAL BERT?


token classification:
- extract the item searched
- NER https://huggingface.co/transformers/v2.2.0/examples.html#named-entity-recognition
- https://github.com/huggingface/transformers/tree/master/examples/token-classification
- https://github.com/huggingface/transformers/blob/master/examples/token-classification/run_ner.py
- https://paperswithcode.com/task/named-entity-recognition-ner
- https://huggingface.co/saibo/legal-roberta-base
- train with Hypothesis


In [45]:
from os import environ, makedirs
from os.path import join, exists, dirname
import requests
from urllib.parse import urlencode
import logging
import csv
import pandas as pd

SEARCH_WARRANT_CSV_FN = 'search_warrants.csv'
API_KEY = environ.get("API_KEY")
STORAGE_PATH = "pdfs/"

/Users/jeremybmerrill/.pyenv/versions/3.8.2/lib/python3.8/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [52]:
def get_pdf(recap_filepath_local, type_of_file="other"):
    logging.basicConfig(level=logging.DEBUG)

    fn = recap_filepath_local.split("/")[-1]
    fp = join(STORAGE_PATH, type_of_file, fn)
    makedirs(dirname(fp), exist_ok=True)
    if exists(fp):
        return fp
    url = recap_filepath_local.replace("/storage", "https://www.courtlistener.com").replace("/sata", "https://www.courtlistener.com")
    with open(fp, 'wb') as f:
        logging.debug(f"actually getting PDF from the web {url}")
        f.write(requests.get(url).content)
    return fp

def get_search_warrant_pdf(recap_filepath_local):
    return get_pdf(recap_filepath_local, "search_warrant")

# def get_docket_entries():
#     "https://www.courtlistener.com/api/rest/v3/docket-entries/?docket__id=XXX"
def search_recap_with_url(url):
    return requests.get(url, headers = {'content-type': 'application/json', "Authorization": f"Token {API_KEY}"}).json()

def search_recap(q=None, description=None, available_only=None, suit_nature=None):
    urlparams = {
        "type": "r", # Document-oriented results from the RECAP Archive
        "available_only": "on" if available_only else "off",
        "order_by": "entry_date_filed desc"
    }
    if suit_nature: urlparams["suitNature"] = suit_nature
    if description: urlparams["description"] = description
    if q: urlparams["q"] = q # wwg1wga
    return search_recap_with_url("https://www.courtlistener.com/api/rest/v3/search/?{}".format(urlencode(urlparams)))
    


## find search warrant docs

In [40]:
def find_search_warrant_documents(n=1000):
#     for each case and document, make a record (in memory or in a DB), so we don\'t duplicate
#     download the documents locally
#    ?q=&type=r&order_by=entry_date_filed%20desc&available_only=on&description=search%20warrant
    next_url = None
    records = []
    while len(records) <= n:
        if len(records) == 0:
            search_result = search_recap(description="search warrant", available_only=True)
            records += search_result["results"]
            next_url = search_result["next"]
        elif next_url:
            search_result = search_recap_with_url(next_url)
            records += search_result["results"]
            next_url = search_result["next"]
        else: # next_url is not None (and it's not the first go)
            break
    return records


In [47]:
search_results = find_search_warrant_documents()

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.courtlistener.com:443
DEBUG:urllib3.connectionpool:https://www.courtlistener.com:443 "GET /api/rest/v3/search/?type=r&available_only=on&order_by=entry_date_filed+desc&description=search+warrant HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.courtlistener.com:443
DEBUG:urllib3.connectionpool:https://www.courtlistener.com:443 "GET /api/rest/v3/search/?available_only=on&description=search+warrant&order_by=entry_date_filed+desc&page=2&type=r HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.courtlistener.com:443
DEBUG:urllib3.connectionpool:https://www.courtlistener.com:443 "GET /api/rest/v3/search/?available_only=on&description=search+warrant&order_by=entry_date_filed+desc&page=3&type=r HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.courtlistener.com:443
DEBUG:urllib3.connectionpool:https://www.courtlistene

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.courtlistener.com:443
DEBUG:urllib3.connectionpool:https://www.courtlistener.com:443 "GET /api/rest/v3/search/?available_only=on&description=search+warrant&order_by=entry_date_filed+desc&page=30&type=r HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.courtlistener.com:443
DEBUG:urllib3.connectionpool:https://www.courtlistener.com:443 "GET /api/rest/v3/search/?available_only=on&description=search+warrant&order_by=entry_date_filed+desc&page=31&type=r HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.courtlistener.com:443
DEBUG:urllib3.connectionpool:https://www.courtlistener.com:443 "GET /api/rest/v3/search/?available_only=on&description=search+warrant&order_by=entry_date_filed+desc&page=32&type=r HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.courtlistener.com:443
DEBUG:urllib3.connectionpool:https://www.co

In [50]:
len(search_results)

1000

In [53]:

with open(SEARCH_WARRANT_CSV_FN, 'w') as csvfile:
    writer = None
    for i, result in enumerate(search_results):
        if i == 0:
            writer = csv.DictWriter(csvfile, fieldnames=result.keys())
            writer.writeheader()
        writer.writerow(result)
        get_search_warrant_pdf(result["filepath_local"])


DEBUG:root:actually getting PDF from the web https://www.courtlistener.com/recap/gov.uscourts.azd.1255791/gov.uscourts.azd.1255791.1.0.pdf
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.courtlistener.com:443
DEBUG:urllib3.connectionpool:https://www.courtlistener.com:443 "GET /recap/gov.uscourts.azd.1255791/gov.uscourts.azd.1255791.1.0.pdf HTTP/1.1" 200 286600
DEBUG:root:actually getting PDF from the web https://www.courtlistener.com/recap/gov.uscourts.miwd.99529/gov.uscourts.miwd.99529.1.1.pdf
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.courtlistener.com:443
DEBUG:urllib3.connectionpool:https://www.courtlistener.com:443 "GET /recap/gov.uscourts.miwd.99529/gov.uscourts.miwd.99529.1.1.pdf HTTP/1.1" 200 127467
DEBUG:root:actually getting PDF from the web https://www.courtlistener.com/recap/gov.uscourts.miwd.99529/gov.uscourts.miwd.99529.1.2.pdf
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.courtlistener.com:443
DEBUG:urlli

DEBUG:urllib3.connectionpool:https://www.courtlistener.com:443 "GET /recap/gov.uscourts.ded.73628/gov.uscourts.ded.73628.1.0.pdf HTTP/1.1" 200 926675
DEBUG:root:actually getting PDF from the web https://www.courtlistener.com/recap/gov.uscourts.mnd.185193/gov.uscourts.mnd.185193.7.0.pdf
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.courtlistener.com:443
DEBUG:urllib3.connectionpool:https://www.courtlistener.com:443 "GET /recap/gov.uscourts.mnd.185193/gov.uscourts.mnd.185193.7.0.pdf HTTP/1.1" 200 44175
DEBUG:root:actually getting PDF from the web https://www.courtlistener.com/recap/gov.uscourts.mied.348318/gov.uscourts.mied.348318.9.0.pdf
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.courtlistener.com:443
DEBUG:urllib3.connectionpool:https://www.courtlistener.com:443 "GET /recap/gov.uscourts.mied.348318/gov.uscourts.mied.348318.9.0.pdf HTTP/1.1" 200 112387
DEBUG:root:actually getting PDF from the web https://www.courtlistener.com/recap/gov.usco

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.courtlistener.com:443
DEBUG:urllib3.connectionpool:https://www.courtlistener.com:443 "GET /recap/gov.uscourts.ded.70787/gov.uscourts.ded.70787.8.0.pdf HTTP/1.1" 200 241389
DEBUG:root:actually getting PDF from the web https://www.courtlistener.com/recap/gov.uscourts.mad.226912/gov.uscourts.mad.226912.3.1.pdf
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.courtlistener.com:443
DEBUG:urllib3.connectionpool:https://www.courtlistener.com:443 "GET /recap/gov.uscourts.mad.226912/gov.uscourts.mad.226912.3.1.pdf HTTP/1.1" 200 371647
DEBUG:root:actually getting PDF from the web https://www.courtlistener.com/recap/gov.uscourts.wvsd.230551/gov.uscourts.wvsd.230551.8.1.pdf
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.courtlistener.com:443
DEBUG:urllib3.connectionpool:https://www.courtlistener.com:443 "GET /recap/gov.uscourts.wvsd.230551/gov.uscourts.wvsd.230551.8.1.pdf HTTP/1.1" 200 13100

DEBUG:root:actually getting PDF from the web https://www.courtlistener.com/recap/gov.uscourts.caed.380844/gov.uscourts.caed.380844.1.0.pdf
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.courtlistener.com:443
DEBUG:urllib3.connectionpool:https://www.courtlistener.com:443 "GET /recap/gov.uscourts.caed.380844/gov.uscourts.caed.380844.1.0.pdf HTTP/1.1" 200 1262388
DEBUG:root:actually getting PDF from the web https://www.courtlistener.com/recap/gov.uscourts.ksd.133371/gov.uscourts.ksd.133371.1.0.pdf
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.courtlistener.com:443
DEBUG:urllib3.connectionpool:https://www.courtlistener.com:443 "GET /recap/gov.uscourts.ksd.133371/gov.uscourts.ksd.133371.1.0.pdf HTTP/1.1" 200 1332587
DEBUG:root:actually getting PDF from the web https://www.courtlistener.com/recap/gov.uscourts.vaed.496708/gov.uscourts.vaed.496708.3.0.pdf
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.courtlistener.com:443
DEBUG:u

DEBUG:urllib3.connectionpool:https://www.courtlistener.com:443 "GET /recap/gov.uscourts.mdd.485844/gov.uscourts.mdd.485844.3.0.pdf HTTP/1.1" 200 658317
DEBUG:root:actually getting PDF from the web https://www.courtlistener.com/recap/gov.uscourts.cacd.796098/gov.uscourts.cacd.796098.3.0.pdf
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.courtlistener.com:443
DEBUG:urllib3.connectionpool:https://www.courtlistener.com:443 "GET /recap/gov.uscourts.cacd.796098/gov.uscourts.cacd.796098.3.0.pdf HTTP/1.1" 200 225313
DEBUG:root:actually getting PDF from the web https://www.courtlistener.com/recap/gov.uscourts.wied.91928/gov.uscourts.wied.91928.2.0.pdf
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.courtlistener.com:443
DEBUG:urllib3.connectionpool:https://www.courtlistener.com:443 "GET /recap/gov.uscourts.wied.91928/gov.uscourts.wied.91928.2.0.pdf HTTP/1.1" 200 626014
DEBUG:root:actually getting PDF from the web https://www.courtlistener.com/recap/gov.u

...

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.courtlistener.com:443
DEBUG:urllib3.connectionpool:https://www.courtlistener.com:443 "GET /recap/gov.uscourts.ohsd.237527/gov.uscourts.ohsd.237527.1.0.pdf HTTP/1.1" 200 2042977
DEBUG:root:actually getting PDF from the web https://www.courtlistener.com/recap/gov.uscourts.vaed.469166/gov.uscourts.vaed.469166.1.0.pdf
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.courtlistener.com:443
DEBUG:urllib3.connectionpool:https://www.courtlistener.com:443 "GET /recap/gov.uscourts.vaed.469166/gov.uscourts.vaed.469166.1.0.pdf HTTP/1.1" 200 1977098
DEBUG:root:actually getting PDF from the web https://www.courtlistener.com/recap/gov.uscourts.vaed.466151/gov.uscourts.vaed.466151.3.0.pdf
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.courtlistener.com:443
DEBUG:urllib3.connectionpool:https://www.courtlistener.com:443 "GET /recap/gov.uscourts.vaed.466151/gov.uscourts.vaed.466151.3.0.pdf HTTP/1.1"

DEBUG:root:actually getting PDF from the web https://www.courtlistener.com/recap/gov.uscourts.casd.669224/gov.uscourts.casd.669224.1.0.pdf
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.courtlistener.com:443
DEBUG:urllib3.connectionpool:https://www.courtlistener.com:443 "GET /recap/gov.uscourts.casd.669224/gov.uscourts.casd.669224.1.0.pdf HTTP/1.1" 200 374389
DEBUG:root:actually getting PDF from the web https://www.courtlistener.com/recap/gov.uscourts.wied.89067/gov.uscourts.wied.89067.1.0.pdf
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.courtlistener.com:443
DEBUG:urllib3.connectionpool:https://www.courtlistener.com:443 "GET /recap/gov.uscourts.wied.89067/gov.uscourts.wied.89067.1.0.pdf HTTP/1.1" 200 305316
DEBUG:root:actually getting PDF from the web https://www.courtlistener.com/recap/gov.uscourts.mad.218074/gov.uscourts.mad.218074.8.1.pdf
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.courtlistener.com:443
DEBUG:urlli

DEBUG:urllib3.connectionpool:https://www.courtlistener.com:443 "GET /recap/gov.uscourts.tned.93561/gov.uscourts.tned.93561.2.0.pdf HTTP/1.1" 200 3193518
DEBUG:root:actually getting PDF from the web https://www.courtlistener.com/recap/gov.uscourts.azd.1198609/gov.uscourts.azd.1198609.31.1.pdf
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.courtlistener.com:443
DEBUG:urllib3.connectionpool:https://www.courtlistener.com:443 "GET /recap/gov.uscourts.azd.1198609/gov.uscourts.azd.1198609.31.1.pdf HTTP/1.1" 200 175663
DEBUG:root:actually getting PDF from the web https://www.courtlistener.com/recap/gov.uscourts.nyed.445136/gov.uscourts.nyed.445136.1.0.pdf
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.courtlistener.com:443
DEBUG:urllib3.connectionpool:https://www.courtlistener.com:443 "GET /recap/gov.uscourts.nyed.445136/gov.uscourts.nyed.445136.1.0.pdf HTTP/1.1" 200 256363
DEBUG:root:actually getting PDF from the web https://www.courtlistener.com/reca

DEBUG:root:actually getting PDF from the web https://www.courtlistener.com/recap/gov.uscourts.ohsd.236882/gov.uscourts.ohsd.236882.1.0.pdf
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.courtlistener.com:443
DEBUG:urllib3.connectionpool:https://www.courtlistener.com:443 "GET /recap/gov.uscourts.ohsd.236882/gov.uscourts.ohsd.236882.1.0.pdf HTTP/1.1" 200 3613714
DEBUG:root:actually getting PDF from the web https://www.courtlistener.com/recap/gov.uscourts.mad.218910/gov.uscourts.mad.218910.4.0.pdf
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.courtlistener.com:443
DEBUG:urllib3.connectionpool:https://www.courtlistener.com:443 "GET /recap/gov.uscourts.mad.218910/gov.uscourts.mad.218910.4.0.pdf HTTP/1.1" 200 764104
DEBUG:root:actually getting PDF from the web https://www.courtlistener.com/recap/gov.uscourts.mad.218910/gov.uscourts.mad.218910.3.1.pdf
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.courtlistener.com:443
DEBUG:urll

DEBUG:urllib3.connectionpool:https://www.courtlistener.com:443 "GET /recap/gov.uscourts.ohsd.193353/gov.uscourts.ohsd.193353.16.0.pdf HTTP/1.1" 200 92379


In [55]:
search_warrants = pd.read_csv(SEARCH_WARRANT_CSV_FN)
search_warrants

,absolute_url,assignedTo,assigned_to_id,attachment_number,attorney,attorney_id,caseName,cause,court,court_citation_string,...,juryDemand,page_count,party,party_id,referredTo,referred_to_id,short_description,snippet,suitNature,timestamp
0,/docket/18601604/4/searchseizure-warrant/,Robert Hardy Cleland,646.0,NaN,NaN,NaN,Search/Seizure Warrant,No cause code entered,"District Court, E.D. Michigan",E.D. Mich.,...,None,3,['Search/Seizure Warrant'],[10226652],NaN,NaN,NaN,\n\n Search/Seizure Warrant Returned Execut...,890 Other Statutory Actions,2020-12-29T08:11:15.659000-08:00
1,/docket/18601939/4/searchseizure-warrant/,Laurie Jill Michelson,2235.0,NaN,NaN,NaN,Search/Seizure Warrant,No cause code entered,"District Court, E.D. Michigan",E.D. Mich.,...,None,3,['Search/Seizure Warrant'],[10226651],NaN,NaN,NaN,\n\n Search/Seizure Warrant Returned Execut...,890 Other Statutory Actions,2020-12-29T08:09:27.475000-08:00
2,/docket/16318335/160/7/united-states-v-parnas/,James Paul Oetken,2463.0,7.0,"['Faith Alison Friedman', 'Gerald B. Lefcourt'...","[6156593, 6156594, 6156595, 6156596, 6156597]",United States v. Parnas,NaN,"District Court, S.D. New York",S.D.N.Y.,...,NaN,29,"['Andrey Kukushkin', 'USA']","[9268253, 9268254]",NaN,NaN,"Exhibit ""G"" - Search Warrant dated May 4, 2020","\n\n DECLARATION of Gerald B. Lefcourt, Esq...",NaN,2020-12-28T03:30:15.759000-08:00
3,/docket/16318335/160/united-states-v-parnas/,James Paul Oetken,2463.0,NaN,"['Faith Alison Friedman', 'Gerald B. Lefcourt'...","[6156593, 6156594, 6156595, 6156596, 6156597]",United States v. Parnas,NaN,"District Court, S.D. New York",S.D.N.Y.,...,NaN,167,"['Andrey Kukushkin', 'USA']","[9268253, 9268254]",NaN,NaN,Declaration in Support of Motion,"\n\n DECLARATION of Gerald B. Lefcourt, Esq...",NaN,2020-12-28T03:30:15.759000-08:00
4,/docket/16318335/160/8/united-states-v-parnas/,James Paul Oetken,2463.0,8.0,"['Faith Alison Friedman', 'Gerald B. Lefcourt'...","[6156593, 6156594, 6156595, 6156596, 6156597]",United States v. Parnas,NaN,"District Court, S.D. New York",S.D.N.Y.,...,NaN,3,"['Andrey Kukushkin', 'USA']","[9268253, 9268254]",NaN,NaN,"Exhibit ""H"" - Andrey Kukushkin's Demand for a ...","\n\n DECLARATION of Gerald B. Lefcourt, Esq...",NaN,2020-12-28T03:30:15.759000-08:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,/docket/16840718/1/united-states-v-734-circle-...,James E. Gates,NaN,NaN,['Sebastian Kielmanovich'],[6123761],"United States v. 734 Circle Drive, Greenville,...",NaN,"District Court, E.D. North Carolina",E.D.N.C.,...,NaN,18,"['USA', '734 Circle Drive, Greenville, NC 27858']","[9212061, 9212062]",NaN,NaN,NaN,\n\n APPLICATION for Search Warrant issued....,NaN,2020-02-13T09:07:20.984000-08:00
996,/docket/16829148/1/united-states-v-search-of-t...,Roanne L. Mann,NaN,NaN,['Matthew R. Galeotti'],[6117830],United States v. Search of the person known as...,NaN,"District Court, E.D. New York",E.D.N.Y,...,NaN,6,['Search of the person known as Octavio Maldon...,"[9204960, 9204961]",NaN,NaN,Application for Search Warrant,\n\n Application for Search Warrant as to S...,NaN,2020-02-11T08:12:02.960999-08:00
997,/docket/16825076/1/united-states-v-6537-61st-a...,William E. Duffin,NaN,NaN,NaN,NaN,United States v. 6537 61st Ave Kenosha WI 53142,NaN,"District Court, E.D. Wisconsin",E.D. Wis.,...,NaN,24,"['6537 61st Ave Kenosha WI 53142', 'USA']","[9202604, 9202605]",NaN,NaN,Application and Affidavit for Search Warrant,\n\n APPLICATION AND AFFIDAVIT for Search W...,NaN,2020-02-10T13:01:22.780000-08:00
998,/docket/16834296/10/united-states-v-gunn/,NaN,NaN,NaN,NaN,NaN,United States v. Gunn,NaN,"District Court, S.D. Georgia",S.D. Ga.,...,NaN,1,NaN,NaN,NaN,NaN,NaN,\n\n ORDER granting 9 Motion to Seal Sear...,NaN,2020-05-26T22:56:20.632999-07:00


In [69]:
with pd.option_context('display.max_colwidth', None):
    display(search_warrants[search_warrants["docketNumber"].str.contains("1:19-mc-00174", case=False)]["absolute_url"].iloc[0:3])

95    /docket/16341802/22/2/in-the-matter-of-the-application-of-the-associated-press-cable-news/
96    /docket/16341802/22/5/in-the-matter-of-the-application-of-the-associated-press-cable-news/
97    /docket/16341802/22/4/in-the-matter-of-the-application-of-the-associated-press-cable-news/
Name: absolute_url, dtype: object

In [14]:
search_result = find_search_warrant_documents()
search_result

{'count': 11251,
 'next': 'https://www.courtlistener.com/api/rest/v3/search/?available_only=on&description=search+warrant&order_by=entry_date_filed+desc&page=2&type=r',
 'previous': None,
 'results': [{'absolute_url': '/docket/18601604/4/searchseizure-warrant/',
   'assignedTo': 'Robert Hardy Cleland',
   'assigned_to_id': 646,
   'attachment_number': None,
   'attorney': None,
   'attorney_id': None,
   'caseName': 'Search/Seizure Warrant',
   'cause': 'No cause code entered',
   'court': 'District Court, E.D. Michigan',
   'court_citation_string': 'E.D. Mich.',
   'court_exact': 'mied',
   'court_id': 'mied',
   'dateArgued': None,
   'dateFiled': '2020-08-04T00:53:00-07:00',
   'dateTerminated': None,
   'description': 'Search/Seizure Warrant Returned Executed on 8/5/2020 for Financial Account from Capital One. [DESIGNATION: 1] (Dydell, Adriana) (Entered: 12/29/2020)',
   'docketNumber': '3:20-mc-50910',
   'docket_absolute_url': '/docket/18601604/searchseizure-warrant/',
   'docket

In [15]:
# docket_numbers = [res["docketNumber"] for res in search_result["results"]]
# docket_numbers

In [37]:
for res in search_result["results"]:
    print(res["docketNumber"])
    print(res["description"])
    get_search_warrant_pdf(res["filepath_local"])
    print()

3:20-mc-50910
Search/Seizure Warrant Returned Executed on 8/5/2020 for Financial Account from Capital One. [DESIGNATION: 1] (Dydell, Adriana) (Entered: 12/29/2020)
pdfs/search_warrant/gov.uscourts.mied.348331.4.0.pdf

2:20-mc-50912
Search/Seizure Warrant Returned Executed on 8/5/2020 for Financial Account from JP Morgan Chase Bank. [DESIGNATION: 1] (Dydell, Adriana) (Entered: 12/29/2020)
pdfs/search_warrant/gov.uscourts.mied.348344.4.0.pdf

1:19-cr-00725
DECLARATION of Gerald B. Lefcourt, Esq. in Support by Andrey Kukushkin as to Lev Parnas, Igor Fruman, Andrey Kukushkin re: 157 MOTION to Sever Defendant Igor Fruman.JOINT MOTION for Relief from the Government's Violation of Attorney-Client Privilege ., 159 MOTION to Sever Defendant Andrey Kukushkin. MOTION for Separate Trial on Counts Andrey Kukushkin (4) Count 4s,4,6s . MOTION to Take Deposition from Foreign National-1 . MOTION to Dismiss based on, inter alia, Breach of the Attorney-Client Privilege. MOTION to Suppress Fruits of Searc